# Video Data EM Notebook

In [203]:
import sys
import py_entitymatching as em
import pandas as pd
import os
import time

###################################################################
#KEY!!!! VARIABLE PREVENTS OVERWRITING LABELED SAMPLED DATA
###################################################################
GENERATE_NEW_LABELED_DATA = False



In [139]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 11:07:58) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

In [146]:
# Get the paths
path_A = os.getcwd() + '/DATA/imdb3.csv'
path_B = os.getcwd() + '/DATA/thenumbers3.csv'
print(path_A)

/Users/dcompgriff/PrivateDocs/College Docs/Graduate School/University of Wisconson/School Docs/Spring 2017/839 Data Analysis/Project Stage 3/DATA/imdb3.csv


In [147]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')
A.head()

,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Toy Story 2,1999,G,92 min,"Animation, Adventure, Comedy",John Lasseter,"Ash Brannon,Lee Unkrich,Tom Hanks,Tim Allen,Joan Cusack,Kelsey Grammer",$245.85M
1,1,The Outlaw Josey Wales,1976,PG,135 min,Western,Clint Eastwood,"Clint Eastwood,Sondra Locke,Chief Dan George,Bill McKinney",$31.80M
2,2,"Monsters, Inc.",2001,G,92 min,"Animation, Adventure, Comedy",Pete Docter,"David Silverman,Lee Unkrich,Billy Crystal,John Goodman,Mary Gibbs,Steve Buscemi",$289.92M
3,3,In the Heat of the Night,1967,Not Rated,110 min,"Crime, Drama, Mystery",Norman Jewison,"Sidney Poitier,Rod Steiger,Warren Oates,Lee Grant",$24.38M
4,4,Chungking Express,1994,PG-13,102 min,"Crime, Drama, Romance",Kar-Wai Wong,"Brigitte Lin,Takeshi Kaneshiro,Tony Chiu-Wai Leung,Faye Wong",$0.60M


In [148]:
B.head()

,id,title,year,mpaa,runtime,genres,director,stars,gross
0,0,Effects,2005,-1,-1,-1,Dusty Nelson,-1,-1
1,1,Ek Haseena Thi Ek Deewana Tha,2017,-1,105 minutes,Drama,Suneel Darshan,-1,"$149,491"
2,2,Ekk Albela,2016,-1,-1,Drama,Shekhar Sartandel,-1,"$1,907"
3,3,The Daisy Chain,2009,R,-1,-1,-1,-1,-1
4,4,Dali & I: The Surreal Story,2011,-1,-1,Drama,Andrew Niccol,Al Pacino,-1


### Check for null entries

In [149]:
B.isnull().sum()

id          0
title       0
year        0
mpaa        0
runtime     0
genres      0
director    0
stars       0
gross       0
dtype: int64

In [150]:
A.isnull().sum()

id          0
title       0
year        0
mpaa        0
runtime     0
genres      0
director    0
stars       0
gross       0
dtype: int64

### Check the data size

In [151]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 4291
Number of tuples in B: 31006
Number of tuples in A X B (i.e the cartesian product): 133046746


In [152]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

In [153]:
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 200, 1, show_progress=False)
len(A1), len(B1)

# But for the purposes of this notebook, we will use the entire table A and B

(140, 200)

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [154]:
# Blocking plan

# A, B -- overlap blocker [title] --------------------|---> candidate set

In [155]:
# Create attribute equivalence blocker
ab = em.OverlapBlocker()

# Block using city attribute
C1 = ab.block_tables(A, B, 'title', 'title', 
                    l_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'], 
                    r_output_attrs=['title', 'year', 'mpaa', 'runtime', 'genres', 'director', 'stars', 'gross'],
                    overlap_size=1
                    )

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:41


In [156]:
len(C1)

10956134

## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10,956,134 (from 133,046,746), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [157]:
# Debug blocker output
startTime = time.time()
dbg = em.debug_blocker(C1, A, B, output_size=200, attr_corres=[('title','title'), ('year', 'year')])
endTime = time.time()
print("Total time: %.2f seconds."%(endTime-startTime))

Total time: 44.68 seconds.


In [158]:
# Display first few tuple pairs from the debug_blocker's output
dbg.head(50)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,rtable_title,rtable_year
0,0,2965,17445,Advise & Consent,1962,Ginger & Rosa,2013
1,1,2965,17842,Advise & Consent,1962,Harry & Son,1984
2,2,2965,17844,Advise & Consent,1962,Harry & Snowman,2016
3,3,2965,19318,Advise & Consent,1962,Lambert & Stamp,2015
4,4,2965,19600,Advise & Consent,1962,David & Layla,2007
5,5,2965,19796,Advise & Consent,1962,Hansel & Gretel,2002
6,6,2965,20161,Advise & Consent,1962,Q & A,1990
7,7,2965,23293,Advise & Consent,1962,Starsky & Hutch,2004
8,8,2965,23439,Advise & Consent,1962,Spring & Arnaud,2016
9,9,2965,25285,Advise & Consent,1962,Town & Country,2001


From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [159]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--

In [160]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C2 = ob.block_candset(C1, 'year', 'year' 
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:37


134228

In [161]:
# Display first two rows from C2
C2.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
34,34,45,3,The Hangover,2009,R,100 min,Comedy,Todd Phillips,"Zach Galifianakis,Bradley Cooper,Justin Bartha,Ed Helms",$277.32M,The Daisy Chain,2009,R,-1,-1,-1,-1,-1
144,144,4286,3,The Loved Ones,2009,R,84 min,"Horror, Thriller",Sean Byrne,"Xavier Samuel,Robin McLeavy,Victoria Thaine,Jessica McNamee",-1,The Daisy Chain,2009,R,-1,-1,-1,-1,-1


In [162]:
# Updated blocking sequence
# A, B ------ overlap blocker [title] -----> C1--
#                                                     
# C1 ------ overlap blocker [year] --------> C2--
#
# C2 ------ overlap blocker [mpaa] --------> C3--

In [163]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C3 = ob.block_candset(C2, 'mpaa', 'mpaa' 
                    )
len(C3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


27767

In [164]:
# Display first two rows from C3
C3.head(2)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
34,34,45,3,The Hangover,2009,R,100 min,Comedy,Todd Phillips,"Zach Galifianakis,Bradley Cooper,Justin Bartha,Ed Helms",$277.32M,The Daisy Chain,2009,R,-1,-1,-1,-1,-1
144,144,4286,3,The Loved Ones,2009,R,84 min,"Horror, Thriller",Sean Byrne,"Xavier Samuel,Robin McLeavy,Victoria Thaine,Jessica McNamee",-1,The Daisy Chain,2009,R,-1,-1,-1,-1,-1


In [326]:
# Create blocker to block tuples that only match based on 'the'.
def overlap_ignoring_words(ltuple, rtuple):
    words = set(['the'])
    # Remove ignore words set from strings.
    lostring = ltuple['title'].lower().split()
    rostring = rtuple['title'].lower().split()
    l_tokens = []
    r_tokens = []
    for word in lostring:
        if word.strip() not in words:
            l_tokens.append(word)
    for word in rostring:
        if word.strip() not in words:
            r_tokens.append(word)
    l_tokens = set(l_tokens)
    r_tokens = set(r_tokens)
        
    # Compute overlap.
    #l_tokens = set(list(map(lambda item: item.strip(), lstring.split())))
    #r_tokens = set(list(map(lambda item: item.strip(), rstring.split())))
    intersection = l_tokens.intersection(r_tokens)
    if len(intersection) >= 1:
        return False
    else:
        return True
    
# Create and apply blocker.
bb = em.BlackBoxBlocker()
bb.set_black_box_function(overlap_ignoring_words)
C4 = bb.block_candset(C3)    
C4.head()
    

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
4146,4146,1757,11,Hamlet,1996,PG-13,242 min,Drama,Kenneth Branagh,"Kenneth Branagh,Julie Christie,Derek Jacobi,Kate Winslet",$4.41M,Hamlet,1996,PG-13,-1,Drama,-1,Kenneth Branagh,"$7,129,670"
6671,6671,1093,20,The Assassination of Jesse James by the Coward Robert Ford,2007,R,160 min,"Biography, Crime, Drama",Andrew Dominik,"Brad Pitt,Casey Affleck,Sam Shepard,Mary-Louise Parker",$3.90M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
6891,6891,1723,20,Sweeney Todd: The Demon Barber of Fleet Street,2007,R,116 min,"Drama, Horror, Musical",Tim Burton,"Johnny Depp,Helena Bonham Carter,Alan Rickman,Timothy Spall",$52.90M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
7619,7619,3795,20,30 Days of Night,2007,R,113 min,"Horror, Thriller",David Slade,"Josh Hartnett,Melissa George,Danny Huston,Ben Foster",$39.57M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
7647,7647,3883,20,In the Valley of Elah,2007,R,121 min,"Crime, Drama, Mystery",Paul Haggis,"Tommy Lee Jones,Charlize Theron,Jonathan Tucker,Jason Patric",$6.78M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1


In [327]:
len(C4)

6334

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 600 tuple pairs for labeling purposes.

In [328]:
# Sample  candidate set
S = em.sample_table(C4, 600)
S.head(50)

,_id,ltable_id,rtable_id,ltable_title,ltable_year,ltable_mpaa,ltable_runtime,ltable_genres,ltable_director,ltable_stars,ltable_gross,rtable_title,rtable_year,rtable_mpaa,rtable_runtime,rtable_genres,rtable_director,rtable_stars,rtable_gross
6891,6891,1723,20,Sweeney Todd: The Demon Barber of Fleet Street,2007,R,116 min,"Drama, Horror, Musical",Tim Burton,"Johnny Depp,Helena Bonham Carter,Alan Rickman,Timothy Spall",$52.90M,Home of the Brave,2007,R,106 minutes,Drama,Irwin Winkler,"Samuel L. Jackson,Jessica Biel,Brian Presley,Curtis ""50 Cent"" Jackson",-1
35988,35988,2769,74,Wristcutters: A Love Story,2006,R,88 min,"Comedy, Drama, Fantasy",Goran Dukic,"Patrick Fugit,Shea Whigham,Tom Waits,Shannyn Sossamon",$0.32M,"Crime, A",2006,R,-1,Drama,-1,-1,-1
36081,36081,2472,74,A Scanner Darkly,2006,R,100 min,"Animation, Crime, Drama",Richard Linklater,"Keanu Reeves,Winona Ryder,Robert Downey Jr.,Rory Cochrane",$5.50M,"Crime, A",2006,R,-1,Drama,-1,-1,-1
52331,52331,3476,116,Boy A,2007,R,102 min,Drama,John Crowley,"Andrew Garfield,Peter Mullan,Shaun Evans,Siobhan Finneran",$0.11M,Butterfly on a Wheel,2007,R,-1,Drama,Mike Barker,-1,-1
76674,76674,2498,154,Mystery Science Theater 3000: The Movie,1996,PG-13,73 min,"Comedy, Horror, Sci-Fi",Jim Mallon,"Trace Beaulieu,Michael J. Nelson,Jim Mallon,Kevin Murphy",$1.01M,Mystery Science Theatre 3000,1996,PG-13,-1,Comedy,-1,-1,-1
76739,76739,3222,160,"Mystery, Alaska",1999,R,119 min,"Comedy, Drama, Sport",Jay Roach,"Russell Crowe,Burt Reynolds,Hank Azaria,Mary McCormack",$8.89M,"Mystery, Alaska",1999,R,119 minutes,Comedy,Jay Roach,"Russell Crowe,Hank Azaria,Mary McCormack,Lolita Davidovich,Ron Eldard,Colm Meaney,Maury Chaykin,...",-1
103328,103328,2296,256,Buried,2010,R,95 min,"Drama, Mystery, Thriller",Rodrigo Cortés,"Ryan Reynolds,José Luis García Pérez,Robert Paterson,Stephen Tobolowsky",$1.03M,Buried,2010,R,94 minutes,Thriller/Suspense,Rodrigo Cortes,Ryan Reynolds,"$21,270,290"
108966,108966,2552,295,Bulworth,1998,R,108 min,"Comedy, Drama, Romance",Warren Beatty,"Warren Beatty,Halle Berry,Kimberly Deauna Adams,Vinny Argiro",$26.53M,Bulworth,1998,R,-1,Comedy,Warren Beatty,Warren Beatty,"$29,203,383"
112898,112898,682,314,Bullets Over Broadway,1994,R,98 min,"Comedy, Crime",Woody Allen,"John Cusack,Dianne Wiest,Jennifer Tilly,Chazz Palminteri",$13.38M,Bullets Over Broadway,1994,R,-1,Comedy,Woody Allen,John Cusack,-1
116862,116862,2486,344,Bugsy,1991,R,136 min,"Biography, Crime, Drama",Barry Levinson,"Warren Beatty,Annette Bening,Harvey Keitel,Ben Kingsley",$49.11M,Bugsy,1991,R,-1,Drama,Barry Levinson,-1,-1


In [272]:
if GENERATE_NEW_LABELED_DATA:
    # Label S interactively. 
    G = em.label_table(S, 'gold')
    #SAVE! TIS A BITCH TO LABEL S AGAIN, AND IT NEEDS TO BE LABELED WITHIN THE UI!
    em.to_csv_metadata(G, './DATA/G.csv')
else:
    G = em.read_csv_metadata('./DATA/G.csv', 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')


In [273]:
#em.to_csv_metadata(G, './DATA/G.csv')
#G2 = em.read_csv_metadata('./G2.csv', 
#                         key='_id',
#                         ltable=A, rtable=B, 
#                         fk_ltable='ltable_id', fk_rtable='rtable_id')

#em.save_table(G, './G2.pkl', metadata_ext='.pklmeta')
#G2 = em.load_table('./G2.pkl', metadata_ext='.pklmeta')
#catalog = em.get_catalog()
#em.show_properties(G2)

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [282]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=5)
I = IJ['train']
J = IJ['test']
print(len(I[I['gold']==1]))
print(len(J[J['gold']==1]))

33
11


## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [283]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [284]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [285]:
# List the names of the features generated
feature_table['feature_name']

0                                 id_id_exm
1                                 id_id_anm
2                            id_id_lev_dist
3                             id_id_lev_sim
4               title_title_jac_qgm_3_qgm_3
5           title_title_cos_dlm_dc0_dlm_dc0
6           title_title_jac_dlm_dc0_dlm_dc0
7                           title_title_mel
8                      title_title_lev_dist
9                       title_title_lev_sim
10                          title_title_nmw
11                           title_title_sw
12                mpaa_mpaa_jac_qgm_3_qgm_3
13            mpaa_mpaa_cos_dlm_dc0_dlm_dc0
14            mpaa_mpaa_jac_dlm_dc0_dlm_dc0
15                            mpaa_mpaa_mel
16                       mpaa_mpaa_lev_dist
17                        mpaa_mpaa_lev_sim
18                            mpaa_mpaa_nmw
19                             mpaa_mpaa_sw
20          runtime_runtime_jac_qgm_3_qgm_3
21      runtime_runtime_cos_dlm_dc0_dlm_dc0
22      runtime_runtime_jac_dlm_

### Converting the development set to feature vectors

In [286]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='gold',
                            show_progress=False)

In [287]:
# Display first few rows
H.head(3)

,_id,ltable_id,rtable_id,id_id_exm,id_id_anm,id_id_lev_dist,id_id_lev_sim,title_title_jac_qgm_3_qgm_3,title_title_cos_dlm_dc0_dlm_dc0,title_title_jac_dlm_dc0_dlm_dc0,...,director_director_lev_sim,director_director_nmw,director_director_sw,gross_gross_lev_dist,gross_gross_lev_sim,gross_gross_jar,gross_gross_jwn,gross_gross_exm,gross_gross_jac_qgm_3_qgm_3,gold
89,1527164,2790,4352,0,0.641085,4,0.0,0.000000,0.000000,0.000000,...,0.250000,-1.0,3.0,5.0,0.166667,0.555556,0.555556,0,0.000000,0
347,6405280,2914,17394,0,0.167529,4,0.2,0.071429,0.218218,0.111111,...,0.153846,1.0,3.0,10.0,0.166667,0.541667,0.633333,0,0.090909,0
472,9006279,649,25385,0,0.025566,5,0.0,1.000000,1.000000,1.000000,...,1.000000,14.0,14.0,10.0,0.166667,0.472222,0.525000,0,0.043478,1


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [288]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query erro

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.910000,0.946667,0.920498
1,RF,0.977778,0.913333,0.937709
2,SVM,0.800000,0.158333,0.258730
3,LinReg,0.977778,0.873333,0.912941
4,LogReg,0.927273,0.875000,0.880702
5,NaiveBayes,0.790729,0.966667,0.858768


### Debugging matcher

In [289]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [290]:
# Debug decision tree using GUI
# em.vis_debug_rf(rf, U, V, 
#        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
#        target_attr='gold')

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [291]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [292]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [293]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [294]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)

Precision : 91.67% (11/12)
Recall : 100.0% (11/11)
F1 : 95.65%
False positives : 1 (out of 12 positive predictions)
False negatives : 0 (out of 168 negative predictions)
